<a href="https://colab.research.google.com/github/silverCore97/Bagua/blob/main/Task3_Ricola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imported modules

In [1]:
import tensorflow as tf
# Seed required to be fixed
tf.random.set_seed(38)

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import tensorflow.compat.v2 as tfv2
import tensorflow_hub as hub

from matplotlib import image
import matplotlib.pyplot as plt
import matplotlib as mpl

import zipfile
import os
import shutil
import random

Mount drive and set required paths. Just load train_triplets.txt, test_triplets.txt and food.zip into your Colab Notebooks folder. Read list below to make sure all required files are in the folder.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

train_triplets_path = '/content/drive/MyDrive/Colab Notebooks/train_triplets.txt'
test_triplets_path = '/content/drive/MyDrive/Colab Notebooks/test_triplets.txt'
train_triplets_path = '/content/drive/MyDrive/Colab Notebooks/train_triplets_custom.txt'
val_triplets_path = '/content/drive/MyDrive/Colab Notebooks/val_triplets_custom.txt'
food_path = '/content/drive/MyDrive/Colab Notebooks/food.zip'
print(os.listdir("/content/drive/MyDrive/Colab Notebooks"))

Mounted at /content/drive
['cnn.ipynb', 'food', 'Model_savings', 'resnet152_Test_3', 'Kopie von Task4_transfer_learning_individual_image_transformation-resnet152.ipynb', 'task4_v6.ipynb', 'Project (1).ipynb', 'Task4.ipynb', 'Project.ipynb', 'Kopie von W2_exercise (2).ipynb', 'Kopie von W2_exercise (1).ipynb', 'Kopie von W2_exercise.ipynb', 'Benchmark test.ipynb', 'BaguaTest.ipynb', 'CNN_3.ipynb', 'ResNet blocks.ipynb', 'Cifar100.ipynb', 'Batch Normalization.ipynb', 'tensorflow basics.ipynb', '卷起来 tensorflow.ipynb', 'APPG.ipynb', 'Untitled0.ipynb', 'Least squares.ipynb', 'Parameter to group server.ipynb', 'Prague.ipynb', 'Efficient Allreduce.ipynb', 'BaguaExampleImagenet.ipynb', 'bagua torch_api contrib fuse optimizer.py.ipynb', 'Task2.ipynb', 'Horovod_new_alg.ipynb', 'qadam.ipynb', 'qslocal.ipynb', 'Untitled1.ipynb', 'Untitled (1)', 'Bagua_Example_MNIST_with_alg.ipynb', 'Untitled2.ipynb', '3D.ipynb', 'Kopie von Task3.ipynb', 'Kopie von colab2.ipynb', 'Task3.ipynb', 'food_270_224_in', '

Code to generate new disjoint training and validation picture sets and their correponding triplets from the original training set.

In [ ]:
# Read triplets as list of tuples
def read_file(filename: str, n=None):
    """
    reads one of the files test_triplets.txt or train_triplets.txt and return a list of tuples (a, b, c)
        as described in the exercise description.
    :param filename:
    :param n: how many lines to load. all if n is empty or None
    :return: list of tuples (a, b, c), where a, b, c are all integers in [0, 9999] padded with 0 s.t len == 5
    """
    with open(filename) as fobj:
        r = [tuple(map(int, line.strip().split(" "))) for i, line in enumerate(fobj)]
        return r[:n] if n else r


# Create train and validation set using disjoint picture sets
def generate_train_val_triplets(n = 1000):
    """
    generete a train_triplets_custom.txt and a val_triplets_custom.txt file, which can then be loaded by
    ImageTripletClass (see helper.py), s.t. the images for validating and training are distinct.
    :param n: how many images approx in the val set
    :return: None
    """
    print("[ * ] Generating triplets")
    triplets = read_file("drive/MyDrive/Colab Notebooks/train_triplets.txt")
    train_triplets = 0
    val_triplets = 0

    val_numbers = {random.randint(0, 4999) for _ in range(n)}

    with open("drive/MyDrive/Colab Notebooks/train_triplets_custom.txt", "w") as train_wobj, open("drive/MyDrive/Colab Notebooks/val_triplets_custom.txt", "w") as val_wobj:
        for a, b, c in triplets:
            if a not in val_numbers and b not in val_numbers and c not in val_numbers:
                train_wobj.write(f"{a} {b} {c}\n")
                train_triplets += 1

            elif a in val_numbers and b in val_numbers and c in val_numbers:
                val_wobj.write(f"{a} {b} {c}\n")
                val_triplets += 1

    print(train_triplets, val_triplets)
    print("[ * ] Finished")


generate_train_val_triplets(1900) 


[ * ] Generating triplets


TESTING: Read these triplets

In [ ]:
train_triplets = pd.read_csv(train_triplets_path, delim_whitespace=True, header=None, names =['anchor','positive','negative'])
test_triplets = pd.read_csv(test_triplets_path, delim_whitespace=True, header=None, names =['anchor','positive','negative'])

VALIDATION: Read these triplets

In [ ]:
train_triplets_path = '/content/drive/MyDrive/Colab Notebooks/train_triplets_custom.txt'
val_triplets_path = '/content/drive/MyDrive/Colab Notebooks/val_triplets_custom.txt'
train_triplets = pd.read_csv(train_triplets_path, delim_whitespace=True, header=None, names =['anchor','positive','negative'])
val_triplets = pd.read_csv(val_triplets_path, delim_whitespace=True, header=None, names =['anchor','positive','negative'])

Extract the food.zip file if you have not done so already.


In [ ]:
if not os.path.exists('/content/drive/MyDrive/Colab Notebooks/food_standard2/food'):
  with zipfile.ZipFile(food_path, 'r') as zip_ref:
      zip_ref.extractall('/content/drive/MyDrive/Colab Notebooks/food_standard2')

print('Extracted images to directory.')
print(len(os.listdir("/content/drive/MyDrive/Colab Notebooks/food_standard2/food")))

Here we preprocess the 10000 food pictures into an equal number of feature vectors. First we reshape the pictures to the required input shape of Inception-Resnet v2.

Then we let it run through the preprocessing from keras for this network. After preprocessing, 
each preprocessed picture is run through the convolutional network to be turned into feature vectors.
We have include_top=False so that the final fully connected layer of the network is not included.
(So we can put our own there.)

In [ ]:
# Needed for input of the pretrained model
input_shape = (299,299,3)

# Generate Dataset
train_res = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/food_standard2/food',
    label_mode=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(299, 299),
    shuffle=False
)

base_model_inception = tf.keras.applications.InceptionResNetV2(pooling='avg',include_top=False)
# base_model_inception = hub.KerasLayer('https://tfhub.dev/google/aiy/vision/classifier/food_V1/1')


# freeze the weights
base_model_inception.trainable=False

# declare input
inputs = Input(shape=input_shape)
x = tf.keras.applications.inception_resnet_v2.preprocess_input(inputs)
outputs = base_model_inception(x)

# obtain model and return
base_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compute features
features = base_model.predict(train_res)

Here we simply copy the original triplets and invert negative and positive picture for each triplet as well setting the label of those triplets to 0 so that the network will not be biased to any label.

Then we stack the original and inverted triplets to receive the combined training triplets.

In [ ]:
df = train_triplets.copy()
df_switched = df[['anchor', 'negative', 'positive']]
df_switched = df_switched.rename(columns={"anchor": "anchor", "negative": "positive", 'positive': 'negative'})

df['label'] = 1
df_switched['label'] = 0

df_combined = pd.concat([df, df_switched])
df_combined = df_combined.sample(frac=1).reset_index(drop=True)

# Discarded method of wrong validation (validation pictures not independent from training pictures)
# df_train, df_val = train_test_split(df_combined, test_size=0.2)

VALIDATION

In [ ]:
val_triplets['label'] = 1

We generate the actual triplet feature vectors here by concatenating the three feature vectors corresponding to each triplet in the triplet DataFrame object. The label of each triplets is stored in an empty labels array to which the label values are appended. In the test case, no labels exists.

In [ ]:
# Create triplet features from train or test triplets
def generate_feature_tensor(features, triplet_df, train=False):
  if train:
    # For correct shape of tensor
    train_tensor = np.zeros((triplet_df.shape[0],3*features.shape[1]))
    labels = np.empty(0)

    for i in range(triplet_df.shape[0]):
      anchor, positive, negative, label = triplet_df.iloc[i,:]

      triplet_features = np.concatenate([features[int(anchor),:],features[int(positive),:],features[int(negative),:]])
      train_tensor[i] = triplet_features
      labels = np.append(labels, label)

    return train_tensor, labels

  else:
    # For correct shape of tensor
    test_tensor = np.zeros((triplet_df.shape[0],3*features.shape[1]))
    for i in range(triplet_df.shape[0]):
      anchor, positive, negative = triplet_df.iloc[i,:]

      triplet_features = np.concatenate([features[int(anchor),:],features[int(positive),:],features[int(negative),:]])
      test_tensor[i] = triplet_features

    return test_tensor

TESTING: Only training feature tensors generated here to save space.

In [ ]:
test_tensor = None
predictions = None
train_tensor, labels = generate_feature_tensor(features, df_combined, train=True)

VALIDATION: Labels exist for both training and validation set

In [ ]:
train_tensor, labels = generate_feature_tensor(features, df_combined, train=True)
val_tensor, val_labels = generate_feature_tensor(features, val_triplets, train=True)

Each concatenated triplet feature vector is now run through multiple layers of a trainable fully connected network ending with a sigmoid activation function. (To keep the final value between 0 and 1)

In [ ]:
input = Input(train_tensor.shape[1:])
x0 = Activation('relu')(input)
x1 = Dropout(0.6)(x0)
x2 = Dense(1600)(x1)
x3 = Activation('relu')(x2)
x4 = Dropout(0.3)(x3)
x5 = Dense(400)(x4)
x6 = Activation('relu')(x5)
x7 = Dropout(0.1)(x6)
x8 = Dense(100)(x7)
x9 = Activation('relu')(x8)
x10 = Dense(25)(x9)
x11 = Activation('relu')(x10)
x12 = Dense(1)(x11)
x13 = Activation('sigmoid')(x12)
model = Model(inputs=input, outputs=x13)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

TESTING: We run the model for 15 epochs

In [ ]:
history = model.fit(x=train_tensor, y=labels, epochs=15, verbose=1)  # Can see progress

VALIDATION: We run the training for 20 epochs, one epoch at a time. After each epoch, we determine the validation accuracy.

Note: Rerunning loop after a few epochs does not change current state of model.

In [ ]:

# Use for validation with custom train/val sets
for i in range(20):  
  history = model.fit(x=train_tensor, y=labels, epochs=1,verbose = 1)
  val_predict=model.predict(val_tensor)
  bool_list =[p <0.5 for p in val_predict]
  int_list = [0 if b else 1 for b in bool_list]
  accuracy_diff=tf.math.reduce_mean(tf.abs(val_labels-int_list))
  accuracy_diff_actual=tf.math.reduce_mean(tf.abs(val_labels-val_predict))
  print("Epoch",i,": Validation accuracy binary value: ",1-accuracy_diff)
  print("Epoch",i,": Validation accuracy actual value: ",1-accuracy_diff_actual)


VALIDATION END

In [ ]:
# Remove unneeded variables by setting them to None
train_tensor = None
history = None

We generate the test concatenated triplet feature vectors now to save space.

In [ ]:
test_tensor = generate_feature_tensor(features, test_triplets, train=False)

In [ ]:
# Features no longer needed
del features

Prediction of test labels according to trained model.

In [ ]:
# Predict
predictions = model.predict(test_tensor)
bool_list =[p <0.5 for p in predictions]
int_list = [0 if b else 1 for b in bool_list]

Write results of the test into 'results.txt'.

In [ ]:
with open('drive/MyDrive/Colab Notebooks/results.txt', 'w') as my_list_file:
    # looping over the each ist element
    for element in int_list:
        # writing to file line by line
        my_list_file.write('%s\n' % element)